## How To Perform a Hahn Echo Interferometry Experiment

### Prerequisites
This guide assumes you have a configured `DeviceSetup` as well as `Qubit` objects with assigned parameters. Please see [Getting Started tutorial](../../../../tutorials/sources/getting_started.ipynb) if you need to create your setup and qubits for the first time. 

You can run this notebook on real hardware in the lab. However, if you don't have the hardware at your disposal, you can also run the notebook "as is" using an emulated session (see below). 

If you are just getting started with the LabOne Q Applications Library, please don't hesitate to reach out to us at info@zhinst.com.

### Background
In this how-to guide, you'll perform a Hahn echo experiment to measure the transversal relaxation of a qubit using the `echo` experiment included in the LabOne Q Applications Library. 

In a previous how-to-guide, we already used Ramsey interferometry (link to ramsey how-to) to measure the transversal relaxation ( $T_2^*$ ). However, the Ramsey experiment is sensitive to quasi-static, low-frequency fluctuations. The Hahn echo protocol is less sensitive to quasi-static noise. 

#### Ramsey protocol

Let's start with a short summary of the Ramsey protocol: 

A $X_{\pi/2}$ (X90) pulse positions the Bloch vector on the equator (Fig. 1a). Typically, the carrier frequency of this pulse is slightly detuned from the qubit frequency. As a result, the Bloch vector will rotate around the z-axis. After waiting for a time $\tau$, a second $X_{\pi/2}$-pulse projects the Bloch vector back on to the z-axis. Repeated measurements are made to take an ensemble averaged estimate of the qubit polarization, as a function of $\tau$. The resulting oscillations feature an approximately exponential decay function with decaytime $T_2^*$. (from https://arxiv.org/pdf/1904.06560)

Here, the “ $^*$ ” indicates that the Ramsey experiment is sensitive to inhomogeneous broadening. That is, it is highly sensitive to quasi-static, low-frequency fluctuations that are constant within one experimental trial, but vary from trial to trial, e.g., due to $1/f$-type noise. These fluctuations cause a dephasing of the qubit state (Fig. 1b) during the time $\tau$. 

#### Hahn echo protocol

Now let's have a look at the Hahn echo protocol, which is less sensitive to quasi-static noise:

We perform the same protocol as for the Ramsey interferometry, but place one extra $Y_\pi$ (Y180) pulse in-between the two $X_{\pi/2}$ pulses (X90). The accumulated dephasing during the first period can be undone (refocused) in the second period by rotating the qubit state by 180 $^\circ$ in-between through an $Y_{\pi}$ pulse. The final $X_{\pi/2}$-pulse projects the Bloch vector back on to the z-axis. In summary the overall quasi-static contributions to dephasing are reduced, leaving an estimate $T_{2e}$ that is less sensitive to inhomogeneous broadening mechanisms.

The pulses are generally chosen to be resonant with the qubit transition for a Hahn echo, since any frequency detuning would be nominally refocused anyway.


![](../../../images/hahn_echo.svg "Hahn echo experiment to refocus quasi-static contributions to dephasing.")

### Imports

You'll start by importing the echo experiment from `laboneq_applications`, as well as `laboneq.simple` and a demo quantum platform containing a qpu and device setup to run in emulation mode. See the [Getting Started tutorial](../../../../tutorials/sources/getting_started.ipynb) for more details about the quantum platform and how to create your experimental setup.

In [ ]:
import numpy as np
from laboneq.contrib.example_helpers.plotting.plot_helpers import plot_simulation
from laboneq.simple import *

from laboneq_applications.experiments import echo
from laboneq_applications.qpu_types.tunable_transmon.demo_qpus import demo_platform

### QPU and Device Setup

You'll generate six qubits with pre-defined parameters, as well as a `Device_Setup` consisting of a SHFQC+, HDAWG, and PQSC. If you already have your own `DeviceSetup` and qubits configured, you'll instead initialize the session using your setup as shown in the [Getting Started tutorial](../../../../tutorials/sources/getting_started.ipynb).

In [ ]:
my_platform = demo_platform(6)

Then, you'll connect to the `Session`. Here we connect to an emulated one:

In [ ]:
session = Session(my_platform.setup)
session.connect(do_emulation=True)

### Create a `FolderStore` for Saving Data

The experiment `Workflows` can automatically save the inputs and outputs of all their tasks to the folder path we specify when instantiating the `FolderStore`. Here, we choose the current working directory.

In [ ]:
# import FolderStore from the `workflow` namespace of LabOne Q, which was imported
# from `laboneq.simple`
from pathlib import Path

folder_store = workflow.logbook.FolderStore(Path.cwd())

We disable saving in this guide. To enable it, simply run `folder_store.activate()`.

In [ ]:
folder_store.deactivate()

### Running the Experiment Workflow

You'll now make the experiment workflow and run. For more details on what experiment workflows are and what tasks they execute, see the [Experiment Workflows tutorial](../../../../tutorials/sources/experiment_workflows.ipynb).

Let's first create the options class for the echo experiment and inspect it using the tree view of the option fields per task:

In [ ]:
options = echo.experiment_workflow.options()
options

Or, alternatively, using the `show_fields` function from the `workflow` namespace of LabOne Q, which was imported from `laboneq.simple`:

In [ ]:
workflow.show_fields(options)

Notice that, unless we change it:

- the experiment is run in `AcquisitionType.INTEGRATION` and `AveragingMode.CYCLIC`, using 1024 averages (`count`)
- the experiment is run on the qubit $|g\rangle \leftrightarrow |e\rangle$ (`transition`)
- calibration traces are added (`use_cal_traces=True`) for the states $|g\rangle$ and $|e\rangle$ (`cal_states`)
- the refocusing pulse is an `x180` pulse (can be changed to any other operation from `qpu.quantum_operations`)
- the analysis workflow will run automatically (`do_analysis=True`)
- the figures produced by the analysis are automatically closed (`close_figures=False`)
- the qubit parameters will not be updated (`update=False`)

Here, let's disable closing the figures:

In [ ]:
options.close_figures(False)

Now we run the experiment workflow on the first two qubits in parallel.

In [ ]:
# our qubits live here in the demo setup:
qubits = my_platform.qpu.qubits

exp_workflow = echo.experiment_workflow(
    session=session,
    qpu=my_platform.qpu,
    qubits=[qubits[0], qubits[1]],
    delays=[np.linspace(0, 10e-6, 31), np.linspace(0, 10e-6, 31)],
    options=options
)

workflow_results = exp_workflow.run()

#### Inspect the Tasks That Were Run

In [ ]:
for t in workflow_results.tasks:
    print(t)

#### Inspect the Output Simulation

You can also inspect the compiled experiment and plot the simulated output:

In [ ]:
compiled_experiment = workflow_results.tasks["compile_experiment"].output
plot_simulation(compiled_experiment, length=50e-6)

#### Inspecting the Source Code of the Pulse-Sequence Creation Task

You can inspect the source code of the `create_experiment` task defined in `echo` to see how the experiment pulse sequence is created using quantum operations. To learn more about the latter, see the Quantum Operations tutorial (add link).

In [ ]:
echo.create_experiment.src

To learn more about how to work with experiment `Workflows`, check out the [Experiment Workflows tutorial](../../../../tutorials/sources/experiment_workflows.ipynb).

Here, let's briefly inspect the analysis-workflow results.

#### Analysis Results

Let's check what tasks were run as part of the analysis workflow:

In [ ]:
analysis_workflow_results = workflow_results.tasks["analysis_workflow"]
for t in analysis_workflow_results.tasks:
    print(t)

We can access the qubit parameters extracted by the analysis from the output of the analysis-workflow:

In [ ]:
from pprint import pprint

pprint(analysis_workflow_results.output)  # noqa: T203

Check out the [Experiment Workflows tutorial](../../../../tutorials/sources/experiment_workflows.ipynb) to see how you can manually update the qubit parameters to these new values, or reset them to the old ones. 

Great! You've now run your echo experiment. Check out other experiments in this manual to keep characterizing your qubits.